In [1]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('L19DB_demo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)

with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)

candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(candidates)")]
contributor_cols = [col[1] for col in cursor.execute("PRAGMA table_info(contributors)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

## Exercises
1. Using an `INNER JOIN`, join the candidates and contributors tables by comparing the `candidate_id` and `candidates_id` columns.  Display your joined table with the columns `contributors.last_name`, `contributors.first_name`, and `candidates.last_name`.
2. Do the same inner join as in the last part, but this time append a `WHERE` clause to select a specific candidate's last name.

In [2]:
query = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name FROM contributors INNER JOIN candidates ON candidates.id = contributors.candidate_id'''
viz_tables(['contributors_last_name', 'contributors_first_name', 'candidates_last_name'], query)

,contributors_last_name,contributors_first_name,candidates_last_name
0,Agee,Steven,Huckabee
1,Ahrens,Don,Huckabee
2,Ahrens,Don,Huckabee
3,Ahrens,Don,Huckabee
4,Akin,Charles,Huckabee
5,Akin,Mike,Huckabee
6,Akin,Rebecca,Huckabee
7,Aldridge,Brittni,Huckabee
8,Allen,John D.,Huckabee
9,Allen,John D.,Huckabee


In [3]:
query = '''SELECT contributors.last_name, contributors.first_name, candidates.last_name FROM contributors INNER JOIN candidates ON candidates.id = contributors.candidate_id WHERE candidates.last_name = "Obama"'''
viz_tables(['contributors_last_name', 'contributors_first_name', 'candidates_last_name'], query)

,contributors_last_name,contributors_first_name,candidates_last_name
0,Buckler,Steve,Obama
1,Buckler,Steve,Obama
2,Buckheit,Bruce,Obama
3,Buckel,Linda,Obama
4,Buckel,Linda,Obama
5,Buckel,Linda,Obama
6,Buck,Thomas,Obama
7,Buck,Jay,Obama
8,Buck,Blaine,Obama
9,Buck,Barbara,Obama


## Exercises
Use the following two tables to do the first two exercises in this section. Table A has the form:

| nA   | attr  | idA |
| :::: | ::::: | ::: |
| s1   | 23    | 0   |
| s2   | 7     | 2   |
| s3   | 15    | 2   |
| s4   | 31    | 0   |

and table B has the form:

| nB   | attr  | idB |
| :::: | ::::: | ::: |
| t1   | 60    | 0   |
| t2   | 14    | 7   |
| t3   | 22    | 2   |

1. Draw the table that would result from a `LEFT JOIN` using table A as the reference and the `id` columns for comparison.
2. Draw the table that would result from a `LEFT JOIN` using table B as the reference and the `id` columns for comparison.
3. Create a new table with the following form:

| average contribution | number of contributors | candidate last name |
| :::::::::::::::::::: | :::::::::::::::::::::: | ::::::::::::::::::: |
|    ...               |   ...                  |    ...              |

   The table should be created using the `LEFT JOIN` clause on the contributors table by joining the candidates table by the `id` column.  The `average contribution` column and `number of contributors` column should be obtained using the `AVG` and `COUNT` `SQL` functions.  Finally, you should use the `GROUP BY` clause on the candidates last name.

- Draw the table that would result from a `LEFT JOIN` using table A as the reference and the `id` columns for comparison.

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     |
| s2   | 7       | t3  | 22     |
| s3   | 15      | t3  | 22     |
| s4   | 31      | t1  | 60     |

- Draw the table that would result from a `LEFT JOIN` using table B as the reference and the `id` columns for comparison.

| nA   | A.attr  | nB  | B.attr |
| :::: | ::::::: | ::: | :::::: |
| s1   | 23      | t1  | 60     | 
| None | NaN     | t2  | 14     |
| s2   | 7       | t3  | 22     |
| s3   | 15      | t3  | 22     |
| s4   | 31      | t1  | 60     |

In [4]:
query = '''SELECT AVG(contributors.amount), COUNT(contributors.id), candidates.last_name FROM contributors LEFT JOIN candidates ON candidates.id = contributors.candidate_id GROUP BY candidates.last_name'''
viz_tables(['average_contribution', 'number_of_contributors', 'candidates_last_name'], query)

,average_contribution,number_of_contributors,candidates_last_name
0,50.6000,25,Clinton
1,-1360.0000,25,Giuliani
2,772.0000,25,Huckabee
3,452.6000,25,McCain
4,472.4356,25,Obama
5,-519.7088,25,Paul
6,156.0000,25,Romney


In [5]:
db.commit()

In [6]:
# db.close()

## Exercises
1. Use `pandas` to populate the contributors table.
2. Query the contributors tables with the following:
  1. List entries where the state is "VA" and the amount is less than $\$400.00$.
  2. List entries where the state is "NULL".
  3. List entries for the states of Texas and Pennsylvania.
  4. List entries where the amount contributed is between $\$10.00$ and $\$50.00$.

In [7]:
# Using pandas naming convention
dfcontr = pd.read_csv("contributors.txt", sep="|")
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


In [8]:
dbp = sqlite3.connect('L19_pandas_DB.sqlite')
csr = dbp.cursor()
csr.execute("DROP TABLE IF EXISTS candidates")
csr.execute("DROP TABLE IF EXISTS contributors")
csr.execute("PRAGMA foreign_keys=1")

csr.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_name TEXT, 
               party TEXT NOT NULL)''')

dbp.commit() # Commit changes to the database

csr.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

dbp.commit()

In [9]:
dfcontr.query("state=='VA' & amount < 400.00")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,NaN,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,NaN,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,NaN,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,NaN,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [10]:
dfcontr[dfcontr.state.isnull()]

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,NaN,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


In [11]:
dfcontr[dfcontr.state.isin(['TX', 'PA'])]

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
65,NaN,BURGERT,RONALD,L.,5723 PLUMTREE DRIVE,NaN,DALLAS,TX,752524926,-1000.00,2008-03-05,22
70,NaN,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
71,NaN,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.00,2008-03-05,22
79,NaN,Muse,Nina,Jo,2915 Toro Canyon Rd,NaN,Austin,TX,78746,-50.00,2008-04-21,32
93,NaN,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.00,2008-04-21,32
94,NaN,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.00,2008-04-21,32
95,NaN,Johnson,Cathleen,E.,1003 Justin Ln Apt 2016,NaN,Austin,TX,787572648,-14.76,2008-04-21,32
132,NaN,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-09-30,35
133,NaN,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,150.00,2007-08-09,35
134,NaN,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,NaN,PEARLAND,TX,775816547,50.00,2007-07-19,35


In [12]:
dfcontr.query("amount > 10.00 & amount < 50.00")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
26,NaN,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
49,NaN,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,NaN,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
140,NaN,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-09-17,35
143,NaN,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-08-06,35
144,NaN,ABEGG,PATRICIA,T.,1862 E. 5150 S.,NaN,SALT LAKE CITY,UT,841176911,25.0,2007-07-10,35
158,NaN,ABBO,PAULINE,MORENCY,10720 JACOB LANE,NaN,WHITE LAKE,MI,483862274,35.0,2008-01-07,37
160,NaN,ABAIR,PETER,NaN,40 EVANS STREET,NaN,WATERTOWN,MA,24722150,25.0,2008-01-09,37


## Exercises
1. Sort the contributors table by `amount` and order in *descending* order.
2. Select the `first_name` and `amount` columns.
3. Select the `last_name` and `first_name` columns and drop duplicates.
4. Count how many there are after the duplicates have been dropped.

In [13]:
dfcontr.sort_values(by='amount', ascending=False)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,NaN,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.00,2007-08-14,20
159,NaN,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.00,2008-01-25,37
15,NaN,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.00,2007-06-12,16
33,NaN,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.00,2007-09-30,20
28,NaN,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.00,2007-08-14,20
21,NaN,Baker,David,NaN,2550 Adamsbrooke Drive,NaN,Conway,AR,72034,2300.00,2007-04-11,16
13,NaN,Altes,R.D.,NaN,8600 Moody Road,NaN,Fort Smith,AR,72903,2300.00,2007-06-21,16
135,NaN,ABRAMOWITZ,NIRA,NaN,411 HARBOR ROAD,NaN,SOUTHPORT,CT,68901376,2300.00,2007-09-14,35
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.00,2007-05-18,16
174,NaN,ABRAHAM,SALEM,A.,P.O. BOX 7,NaN,CANADIAN,TX,790140007,1300.00,2008-01-30,37


In [14]:
dfcontr[['first_name', 'amount']]

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0
5,Mike,1500.0
6,Rebecca,500.0
7,Brittni,250.0
8,John D.,1000.0
9,John D.,1300.0


In [15]:
dfcontr[['last_name', 'first_name']].drop_duplicates()

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca
7,Aldridge,Brittni
8,Allen,John D.
10,Allison,John W.
11,Allison,Rebecca
13,Altes,R.D.


In [16]:
dfcontr[['last_name', 'first_name']].drop_duplicates().count()

last_name     126
first_name    126
dtype: int64

## Exercises
1. Create a name column for the contributors table with field entries of the form "last name, first name"
2. For contributors from the state of "PA", change the name to "X".
3. Delete the newly created name column.

In [17]:
dfcontr['name'] = dfcontr['last_name'] + ", " + dfcontr['first_name']
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,"Agee, Steven"
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,"Ahrens, Don"
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,"Ahrens, Don"
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,"Ahrens, Don"
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,"Akin, Charles"
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16,"Akin, Mike"
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16,"Akin, Rebecca"
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16,"Aldridge, Brittni"
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16,"Allen, John D."
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16,"Allen, John D."


In [18]:
dfcontr.loc[dfcontr.state == "PA", "name"] = "X"
dfcontr.query("state == 'PA'")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
70,NaN,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
71,NaN,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,NaN,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22,X
93,NaN,Raught,Philip,M,4714 Plum Way,NaN,Pittsburgh,PA,15201,-1046.0,2008-04-21,32,X
94,NaN,Ferrara,Judith,D,1508 Waterford Road,NaN,Yardley,PA,19067,-1100.0,2008-04-21,32,X
165,NaN,ABEL,JOHN,H.,422 THOMAS STREET,NaN,BETHLEHEM,PA,180153316,200.0,2008-01-22,37,X


In [19]:
del dfcontr['name']
dfcontr

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16
5,NaN,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500.0,2007-05-18,16
6,NaN,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500.0,2007-05-18,16
7,NaN,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250.0,2007-06-06,16
8,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000.0,2007-06-11,16
9,NaN,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300.0,2007-06-29,16


## Exercise
Use the `describe()` method on the `contributors` table.

In [20]:
dfcontr.describe()

,id,zip,amount,candidate_id
count,0.0,1.750000e+02,175.000000,175.000000
mean,NaN,3.780014e+08,3.418114,28.000000
std,NaN,3.628278e+08,1028.418999,7.823484
min,NaN,2.474000e+03,-2592.000000,16.000000
25%,NaN,9.336700e+04,-175.000000,20.000000
50%,NaN,3.233313e+08,100.000000,32.000000
75%,NaN,7.816946e+08,300.000000,35.000000
max,NaN,9.951532e+08,4600.000000,37.000000


In [21]:
dbp.commit()

In [22]:
dbp.close()